In [1]:
!git clone https://github.com/Intelligent-Systems-Phystech/2022-Project-94

Cloning into '2022-Project-94'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 547 (delta 4), reused 4 (delta 4), pack-reused 540
Receiving objects: 100% (547/547), 1.94 GiB | 36.02 MiB/s, done.
Resolving deltas: 100% (160/160), done.
Checking out files: 100% (308/308), done.


Vizualize the data

In [1]:
import sys
import os
import HailProject.code.src.data_processing as dp
from HailProject.code.src.utils import *
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
import torch
from HailProject.code.model.losses import EVL
from HailProject.code.model.HailNet import HailNet, train
from torch.optim import Adam, SGD

In [ ]:
feature_names = ['snowmelt']
numpys_26 = dp.get_nps(feature_names, "HailProject/code/data/Hail/2021-5-26/*.tif") 
numpys_25 = dp.get_nps(feature_names, "HailProject/code/data/Hail/2021-5-25/*.tif") 
tmean_26 = np.nan_to_num(numpys_26['snowmelt'])
tmean_25 = np.nan_to_num(numpys_25['snowmelt'])
print("2021-5-26")
print("Number of dimensions: ", len(tmean_26.shape))
print("First dimension: time - ", tmean_26.shape[0], " - hours")
print("Second dimension: longtitude - ", tmean_26.shape[1], " - pixels (points)")
print("Third dimension: latitude - ", tmean_26.shape[2], " - pixels (points)")
print("2021-5-25")
print("Number of dimensions: ", len(tmean_25.shape))
print("First dimension: time - ", tmean_25.shape[0], " - hours")
print("Second dimension: longtitude - ", tmean_25.shape[1], " - pixels (points)")
print("Third dimension: latitude - ", tmean_25.shape[2], " - pixels (points)")
for i in range(12):
    plt.figure(figsize=(10, 5))
    plt.imshow(tmean_25[i], cmap='hot', interpolation='nearest')
    plt.show()

for i in range(12):
    plt.figure(figsize=(10, 5))
    plt.imshow(tmean_26[i], cmap='hot', interpolation='nearest')
    plt.show()

In [ ]:
#tmean_26[i] = np.nan_to_num(tmean_26[i])
print(tmean_26[i])

NameError: ignored

In [2]:
tdl, x = dp.get_traindl(feature_names = [
  'dewpoint_temperature_2m',
  'runoff', 
  #'snowmelt',
  #'snowfall', 
  #'surface_latent_heat_flux',
  #'potential_evaporation',
  #'soil_temperature_level_1',
  #'surface_net_solar_radiation', 
  #'surface_solar_radiation_downwards'
  ],
  eco = False)

In [ ]:
torch.isnan(x).any()

tensor(False)

In [ ]:
from torchvision import transforms
m1 = torch.nn.LayerNorm([359, 533], elementwise_affine=True)
img_tr = x[0, 0, 2]
mean, std = img_tr.float().mean(), img_tr.float().std()
trans = transforms.Normalize(mean, std)
m1(x[0, 0, 2].float()), trans(img_tr.float().unsqueeze(dim=0)), img_tr

In [3]:

num_epochs = 30
n = 2
long = 359
lat = 533
gru_hidden_size = 16 * 16
gru_hidden_layers = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HailNet(n, long, lat, gru_hidden_size, gru_hidden_layers)
loss_fn = torch.nn.BCELoss()
loss_fn.to(device)
model = model.cuda()
opt_adam = Adam(model.parameters(), lr=1e-3)
losses_adam = train(num_epochs, model, loss_fn, opt_adam, tdl)

Epoch: 0 | Loss: 0.5180610418319702
Epoch: 1 | Loss: 0.23219221830368042
Epoch: 2 | Loss: 0.11783215403556824
Epoch: 3 | Loss: 0.06966357678174973
Epoch: 4 | Loss: 0.03808615729212761
Epoch: 5 | Loss: 0.03137841075658798
Epoch: 6 | Loss: 0.025251537561416626
Epoch: 7 | Loss: 0.020683802664279938
Epoch: 8 | Loss: 0.019836075603961945
Epoch: 9 | Loss: 0.01636769436299801
Epoch: 10 | Loss: 0.011918064206838608
Epoch: 11 | Loss: 0.013581623323261738
Epoch: 12 | Loss: 0.011093659326434135
Epoch: 13 | Loss: 0.010299069806933403
Epoch: 14 | Loss: 0.009989659301936626
Epoch: 15 | Loss: 0.00876979622989893
Epoch: 16 | Loss: 0.009296219795942307
Epoch: 17 | Loss: 0.007380680181086063
Epoch: 18 | Loss: 0.006713383831083775
Epoch: 19 | Loss: 0.006953416857868433
Epoch: 20 | Loss: 0.006354830227792263
Epoch: 21 | Loss: 0.007250810973346233
Epoch: 22 | Loss: 0.006951239425688982
Epoch: 23 | Loss: 0.005737972445785999
Epoch: 24 | Loss: 0.004577300511300564
Epoch: 25 | Loss: 0.004750106483697891
Epoch

In [4]:
from sklearn.metrics import accuracy_score, classification_report
from HailProject.code.model.HailNet import test
#num_epochs = 30
#n = 1
#long = 359
#lat = 533
#gru_hidden_size = 16 * 16
#gru_hidden_layers = 1
#model = HailNet(n, long, lat, gru_hidden_size, gru_hidden_layers)
#model = model.cuda()
preds, trues = test(model, tdl, ["accuracy"], {"accuracy": accuracy_score})
print(preds)
preds = torch.cat(preds)
trues = torch.cat(trues)
threshold = 0.5
for i in range(preds.shape[0]):
    if preds[i].item() < threshold:
        preds[i] = torch.Tensor([0])

[tensor([[0.9954],
        [0.9945],
        [0.0060],
        [0.9954]], device='cuda:0'), tensor([[0.9958],
        [0.0033],
        [0.9961],
        [0.9963]], device='cuda:0'), tensor([[0.0047],
        [0.0051],
        [0.0047],
        [0.9966]], device='cuda:0'), tensor([[0.0036],
        [0.9961],
        [0.0042],
        [0.0060]], device='cuda:0'), tensor([[0.0037],
        [0.0031],
        [0.0043],
        [0.0040]], device='cuda:0'), tensor([[0.9956],
        [0.9966]], device='cuda:0')]


In [5]:
threshold = 0.5
for i in range(preds.shape[0]):
    if preds[i] < threshold:
        preds[i] = 0
    else:
        preds[i] = 1
print(classification_report(trues.cpu().numpy(), preds.cpu().numpy().squeeze(), target_names=["No Hail", "Hail"]))

              precision    recall  f1-score   support

     No Hail       1.00      1.00      1.00        12
        Hail       1.00      1.00      1.00        10

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22



In [ ]:
trues.cpu().numpy()


array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
preds.cpu().numpy().squeeze()

array([1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 1.], dtype=float32)

In [ ]:
for name, param in model.named_parameters():
    print(name, param)

embedding_layer.lin1.weight Parameter containing:
tensor([[-8.5990e-04,  1.2121e-03, -7.1599e-04,  ...,  1.1333e-03,
          1.2722e-03,  7.1235e-04],
        [-1.2438e-03, -1.2515e-03,  1.1208e-04,  ...,  7.9679e-04,
         -8.2510e-04,  5.6878e-04],
        [ 9.8738e-05,  8.8378e-04, -9.0103e-04,  ..., -1.2997e-03,
         -1.1778e-03,  6.4279e-04],
        ...,
        [-4.4698e-04, -4.9592e-04, -4.7572e-05,  ..., -5.5030e-04,
         -2.1388e-04,  4.4639e-05],
        [-3.0805e-04, -1.4338e-04,  6.1508e-05,  ...,  4.1453e-04,
          2.4099e-04,  1.2583e-03],
        [ 8.6174e-04,  8.1952e-04, -5.2801e-04,  ...,  1.3673e-05,
          2.9490e-04,  1.0164e-03]], device='cuda:0', requires_grad=True)
embedding_layer.lin1.bias Parameter containing:
tensor([-2.8098e-04, -1.8797e-04,  9.7263e-04,  3.5260e-04,  8.8769e-04,
        -4.2013e-04, -1.8682e-04,  6.3403e-04,  6.7820e-05, -1.0204e-03,
         1.4798e-04,  8.0847e-04, -7.0003e-04, -9.5292e-05,  2.1170e-04,
         3.175

In [8]:
tdl, x = dp.get_traindl(feature_names = [
  'dewpoint_temperature_2m',
  'runoff', 
  #'snowmelt',
  #'snowfall', 
  #'surface_latent_heat_flux',
  #'potential_evaporation',
  #'soil_temperature_level_1',
  #'surface_net_solar_radiation', 
  #'surface_solar_radiation_downwards'
  ],
  eco = False,
  train = False)

In [9]:
#model = HailNet(n, long, lat, gru_hidden_size, gru_hidden_layers)
#model = model.cuda()
preds, trues = test(model, tdl, ["accuracy"], {"accuracy": accuracy_score})
print(preds)
preds = torch.cat(preds)
trues = torch.cat(trues)
threshold = 0.5
for i in range(preds.shape[0]):
    if preds[i].item() < threshold:
        preds[i] = torch.Tensor([0])

[tensor([[0.0041],
        [0.9961],
        [0.9962],
        [0.9960]], device='cuda:0'), tensor([[0.0059],
        [0.9944],
        [0.0045]], device='cuda:0')]


In [10]:
threshold = 0.5
for i in range(preds.shape[0]):
    if preds[i] < threshold:
        preds[i] = 0
    else:
        preds[i] = 1
print(classification_report(trues.cpu().numpy(), preds.cpu().numpy().squeeze(), target_names=["No Hail", "Hail"]))

              precision    recall  f1-score   support

     No Hail       1.00      1.00      1.00         3
        Hail       1.00      1.00      1.00         4

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

